# Lab 2 

Let's get started on Lab 2! We will be diving into the fundamentals of analyzing text and covering the following topics:

1. Zipf and Hapax graphs
2. Ngrams and text pre-processing, tokenization, featurization, collocation
3. Observing differences between text using the scattertext library

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/ls-190colab/cap-examples
%ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/ls-190colab/cap-examples
api_text_search/          get_judges/             python_wrapper/
api_to_csv/               introduction/           README.md
api_tutorial_downloader/  labelling_summarizing/  requirements.in
bulk_exploration/         LICENSE                 requirements.txt
bulk_extract/             logs.log                sample_text.txt
config/                   map_courts/             settings.py
d2v-vectors.pkl           m.model                 tests/
data/                     m.vocab                 utils.py
fabfile.py                ngrams/                 w2v-vectors.pkl
full_text_search/         __pycache__/            X_tfidf.pkl


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install shifterator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install flashtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip-compile --output-file=requirements.txt requirements.in
!pip install config

#
# This file is autogenerated by pip-compile with python 3.7
# To update, run:
#
#    pip-compile --output-file=requirements.txt requirements.in
#
--extra-index-url https://us-python.pkg.dev/colab-wheels/public/simple/

argon2-cffi==21.3.0
    # via notebook
argon2-cffi-bindings==21.2.0
    # via argon2-cffi
attrs==21.4.0
    # via
    #   jsonschema
    #   pytest
backcall==0.2.0
    # via ipython
bcrypt==3.2.0
    # via paramiko
bleach==4.1.0
    # via nbconvert
blis==0.7.6
    # via
    #   spacy
    #   thinc
catalogue==2.0.6
    # via
    #   spacy
    #   srsly
    #   thinc
certifi==2021.10.8
    # via requests
cffi==1.15.0
    # via
    #   argon2-cffi-bindings
    #   bcrypt
    #   cryptography
    #   pynacl
charset-normalizer==2.0.12
    # via requests
click==8.0.3
    # via
    #   pip-tools
    #   typer
cryptography==36.0.1
    # via paramiko
cssselect==1.1.0
    # via pyquery
cycler==0.11.0
    # via matplotlib
cymem==2.0.6
    # via
    #   preshed
    #   spacy
    #

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import os
import sys
sys.path.append('..')

import lzma
import json
import re
import itertools
import collections
import pandas as pd


import os
import sys
sys.path.append('..')

import lzma
import json

from config import settings_base as settings
import utils

Load the Data
In this lab, we will be investing cases in Delaware. To extract cases, we use the case.Law API

In [ ]:
compressed_file = utils.get_cases_from_bulk(jurisdiction="Delaware", data_format="json")

downloading Delaware-20200302-text.zip into ../data dir


57900it [00:02, 22563.49it/s]


extracting Delaware-20200302-text.zip into ../data dir
Done.


In [ ]:
cases = []
print("File path:", compressed_file)
with lzma.open(compressed_file) as infile:
    for line in infile:
        record = json.loads(str(line, 'utf-8'))
        cases.append(record)

print("Case count: %s" % len(cases))

File path: /content/gdrive/MyDrive/ls-190colab/cap-examples/config/../data/Delaware-20200302-text/data/data.jsonl.xz
Case count: 16772


In [ ]:
df = pd.DataFrame(cases)
df.head()

,id,url,name,name_abbreviation,decision_date,docket_number,first_page,last_page,citations,volume,reporter,court,jurisdiction,frontend_url,preview,casebody
0,11737169,https://api.capapi.org/v1/cases/11737169/,In the Matter of Ronald L. CANTRELL for a Writ...,In re Cantrell,1996-07-01,"No. 258, 1996",525,526,"[{'type': 'official', 'cite': '678 A.2d 525'}]",{'url': 'https://api.capapi.org/v1/volumes/320...,{'url': 'https://api.capapi.org/v1/reporters/2...,{'url': 'https://api.capapi.org/v1/courts/del-...,"{'name': 'Del.', 'name_long': 'Delaware', 'whi...",https://cite.capapi.org/a2d/678/525/,[],"{'data': {'judges': ['Before WALSH, HOLLAND, a..."
1,11684804,https://api.capapi.org/v1/cases/11684804/,"Hendrik LIKET, Defendant Below, Appellant, v. ...",Liket v. State,1998-11-04,"No. 286, 1997",935,939-941,"[{'type': 'official', 'cite': '719 A.2d 935'}]",{'url': 'https://api.capapi.org/v1/volumes/320...,{'url': 'https://api.capapi.org/v1/reporters/2...,{'url': 'https://api.capapi.org/v1/courts/del-...,"{'name': 'Del.', 'name_long': 'Delaware', 'whi...",https://cite.capapi.org/a2d/719/935/,[],"{'data': {'judges': ['Before VEASEY, C.J., WAL..."
2,11684732,https://api.capapi.org/v1/cases/11684732/,"STATE of Delaware, Employer Below, Appellant, ...",State v. Steen,1998-10-22,"No. 57, 1998",930,935,"[{'type': 'official', 'cite': '719 A.2d 930'}]",{'url': 'https://api.capapi.org/v1/volumes/320...,{'url': 'https://api.capapi.org/v1/reporters/2...,{'url': 'https://api.capapi.org/v1/courts/del-...,"{'name': 'Del.', 'name_long': 'Delaware', 'whi...",https://cite.capapi.org/a2d/719/930/,[],"{'data': {'judges': ['Before VEASEY, C.J., WAL..."
3,11637715,https://api.capapi.org/v1/cases/11637715/,"Eric BARKLEY, Defendant Below, Appellant, v. S...",Barkley v. State,1999-02-19,"No. 116, 1998",558,561,"[{'type': 'official', 'cite': '724 A.2d 558'}]",{'url': 'https://api.capapi.org/v1/volumes/320...,{'url': 'https://api.capapi.org/v1/reporters/2...,{'url': 'https://api.capapi.org/v1/courts/del-...,"{'name': 'Del.', 'name_long': 'Delaware', 'whi...",https://cite.capapi.org/a2d/724/558/,[],"{'data': {'judges': ['Before VEASEY, C.J., WAL..."
4,11638241,https://api.capapi.org/v1/cases/11638241/,"S&R ASSOCIATES, L.P., III, a Limited Partnersh...","S&R Associates, L.P. v. Shell Oil Co.",1998-09-30,No. 94C-06-189-WTQ,431,440,"[{'type': 'official', 'cite': '725 A.2d 431'}]",{'url': 'https://api.capapi.org/v1/volumes/320...,{'url': 'https://api.capapi.org/v1/reporters/2...,{'url': 'https://api.capapi.org/v1/courts/del-...,"{'name': 'Del.', 'name_long': 'Delaware', 'whi...",https://cite.capapi.org/a2d/725/431/,[],"{'data': {'judges': [], 'attorneys': ['Stephan..."


In [ ]:
# Helper function to extract the main text in casebody
def get_text(x):
    if len(x['data']['opinions'])>0:
        return x['data']['opinions'][0]['text']
    else:
        return 0

# Helper function to remove punctuations in text
def remove_punctuation(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [ ]:
df['court_name'] = df["court"].apply(lambda x:x['name'])
chancery = df[df['court_name']=='Delaware Court of Chancery']
chancery = chancery.sample(275)
common_pleas = df[df['court_name']=='Delaware Court of Common Pleas']
concat = pd.concat([chancery,common_pleas])
concat['case_text'] = concat['casebody'].apply(lambda x:get_text(x))
concat = concat[concat['case_text']!=0]
concat['case_text'] = concat['case_text'].apply(lambda x:remove_punctuation(x))

In [ ]:
concat.head()

,id,url,name,name_abbreviation,decision_date,docket_number,first_page,last_page,citations,volume,reporter,court,jurisdiction,frontend_url,preview,casebody,court_name,case_text
9100,1870605,https://api.capapi.org/v1/cases/1870605/,"Pierre S. duPont, vs. Wilmington Trust Company...",duPont v. Wilmington Trust Co.,1946-01-16,,7,30,"[{'type': 'official', 'cite': '29 Del. Ch. 7'}]",{'url': 'https://api.capapi.org/v1/volumes/320...,{'url': 'https://api.capapi.org/v1/reporters/2...,{'url': 'https://api.capapi.org/v1/courts/del-...,"{'name': 'Del.', 'name_long': 'Delaware', 'whi...",https://cite.capapi.org/del-ch/29/7/,[],"{'data': {'judges': [], 'attorneys': ['Clarenc...",Delaware Court of Chancery,Harrington ChancellorPierre S duPont seeks to ...
3358,460959,https://api.capapi.org/v1/cases/460959/,"James Bradford Company, vs. United Leather Com...",James Bradford Co. v. United Leather Co.,1914-09-14,,46,53,"[{'type': 'official', 'cite': '11 Del. Ch. 46'}]",{'url': 'https://api.capapi.org/v1/volumes/320...,{'url': 'https://api.capapi.org/v1/reporters/2...,{'url': 'https://api.capapi.org/v1/courts/del-...,"{'name': 'Del.', 'name_long': 'Delaware', 'whi...",https://cite.capapi.org/del-ch/11/46/,[],"{'data': {'judges': [], 'attorneys': ['Richard...",Delaware Court of Chancery,The ChancellorThe questions of law raised in t...
10465,8080445,https://api.capapi.org/v1/cases/8080445/,"EASTERN SHORE NATURAL GAS COMPANY, a Delaware ...",Eastern Shore Natural Gas Co. v. Stauffer Chem...,1971-12-16,,826,829,"[{'type': 'official', 'cite': '285 A.2d 826'}]",{'url': 'https://api.capapi.org/v1/volumes/320...,{'url': 'https://api.capapi.org/v1/reporters/2...,{'url': 'https://api.capapi.org/v1/courts/del-...,"{'name': 'Del.', 'name_long': 'Delaware', 'whi...",https://cite.capapi.org/a2d/285/826/,[],"{'data': {'judges': [], 'attorneys': ['John J....",Delaware Court of Chancery,MARVEL Vice ChancellorPlaintiff seeks the foll...
8052,477491,https://api.capapi.org/v1/cases/477491/,In the Matter of the Petition of Horace B. Jam...,In re James,1927-01-12,,253,255,"[{'type': 'official', 'cite': '15 Del. Ch. 253'}]",{'url': 'https://api.capapi.org/v1/volumes/320...,{'url': 'https://api.capapi.org/v1/reporters/2...,{'url': 'https://api.capapi.org/v1/courts/del-...,"{'name': 'Del.', 'name_long': 'Delaware', 'whi...",https://cite.capapi.org/del-ch/15/253/,[],"{'data': {'judges': [], 'attorneys': ['James M...",Delaware Court of Chancery,The ChancellorThe section of the Code Revised ...
6250,484206,https://api.capapi.org/v1/cases/484206/,"Wilmington Trust Company, a corporation of the...",Wilmington Trust Co v. Grier,1932-07-27,,34,45,"[{'type': 'official', 'cite': '19 Del. Ch. 34'}]",{'url': 'https://api.capapi.org/v1/volumes/320...,{'url': 'https://api.capapi.org/v1/reporters/2...,{'url': 'https://api.capapi.org/v1/courts/del-...,"{'name': 'Del.', 'name_long': 'Delaware', 'whi...",https://cite.capapi.org/del-ch/19/34/,[],"{'data': {'judges': [], 'attorneys': ['Charles...",Delaware Court of Chancery,The Chancellor Whether a power has been execut...


# Part 1 

**Zipf and the Hapax Legomenon**

The Zipfian Graph
George Zipf was an American linguist, and is well-known for his creation of the Zipfian graph. This graph details the relationship between a word's ranking and its frequency within a corpus.

Here, we'll be exploring this phenomenon with our own Zipfian graph using the case data we loaded above!

In [ ]:
# Here, we're indexing through case data for the first example of text.
case_body = df['casebody'][0]['data']['opinions'][0]['text']
tokenized_body = nltk.word_tokenize(case_body)

# The FreqDist object acts similarly to a dictionary in this scenario.
case_freq_dist = nltk.FreqDist()

# For each token in the case text, we are incrementing its frequency value by 1.
for token in tokenized_body:
  case_freq_dist[token] += 1

NameError: ignored

The matplotlib library is a great way to visualize the data you've collected!

In [ ]:
# Replace the ellipses following plt.scatter to map the token rankings 
# on the x-axis against the frequency values on the y-axis.

import matplotlib.pyplot as plt

# Gathering the keys and values to plot.
token_rankings = range(len(case_freq_dist.keys()))
freq_vals = case_freq_dist.values()

def plot_ranking_frequency():
  # Here, we create a scatterplot with the token (word) rankings and frequencies.
  # HINT: Use variables already defined for you!
  plt.scatter(x = token_rankings,y = freq_vals)

  # This is modifying the settings of the graph to add labels and adjust the scale.
  plt.locator_params(axis="both", integer=True)
  plt.xlabel('Token Ranking')
  plt.ylabel('Token Frequency')
  plt.title('Token Ranking vs. Frequency')

plot_ranking_frequency()

Notice any trends in the data? Does the data look linear?

A key takeaway from a text's Zipfian graph is that almost always, the rankings of the token has a logarithmic relationship with its frequency.

Now, let's go ahead and map a token's ranking against the logarithmic transformation of its frequency. This linearizes the relationship between the two variables.

In [ ]:
plot_ranking_frequency()

# We're changing the scale of the y-axis(frequency) to logarithmic.
plt.yscale(value = 'log')

The Hapax Legomenon
Just because a word is mentioned frequently in a text, does it mean it's important? Here are the top 5 words in the English language, ranked based on average use.

the
be
to
of
and
Clearly, these words would not be able to give us any insightful meaning into a text purely by knowing they have high frequency rankings.

This logic also applies in the other direction: just because a word is mentioned only once, does the word have very little importance? A word appearing only once in a text is called a hapax legomenon.

The conversation surrounding guns in the context of legislature has been extremely controversial in the last decade. However, "arms" is mentioned only one time in the entire U.S. Constitution.

It is very important to keep in mind that a word's importance in meaning to a text is not always related to its ranking in frequency, also known as a frequentist approach to text.

# Part 2 - Ngrams and text pre-processing, tokenization, featurization, collocation

In [ ]:
opinion_data = []
for case in cases:
    for opinion in case["casebody"]["data"]["opinions"]:
        temp = {}
        keys = list(case.keys())
        keys.remove('casebody')
        for key in keys:         
            temp[key] = case[key]
        keys = list(opinion.keys())
        for key in keys:         
            temp[key] = opinion[key]
        opinion_data.append(temp)
        
df = pd.DataFrame(opinion_data)
df["citations"] = df["citations"].apply(lambda x:x[0]['cite'])
df["court"] = df["court"].apply(lambda x:x['name'])
df["decision_date"] = df["decision_date"].apply(lambda x:int(x[:4]))
df = df.drop(["docket_number", "first_page", 
                                "last_page", "name_abbreviation",
                                "reporter", "volume", "jurisdiction"], axis=1)
df = df[["id", "name", "decision_date", "court", "citations", "author", "type", "text"]]

df.head()

,id,name,decision_date,court,citations,author,type,text
0,11737169,In the Matter of Ronald L. CANTRELL for a Writ...,1996,Delaware Supreme Court,678 A.2d 525,"HOLLAND, Justice:",majority,"HOLLAND, Justice:\nThe Court has before it Ron..."
1,11684804,"Hendrik LIKET, Defendant Below, Appellant, v. ...",1998,Delaware Supreme Court,719 A.2d 935,PER CURIAM.,majority,"PER CURIAM.\nIn this appeal, we affirm the tri..."
2,11684732,"STATE of Delaware, Employer Below, Appellant, ...",1998,Delaware Supreme Court,719 A.2d 930,"HOLLAND, Justice:",majority,"HOLLAND, Justice:\nThis is an appeal by the em..."
3,11637715,"Eric BARKLEY, Defendant Below, Appellant, v. S...",1999,Delaware Supreme Court,724 A.2d 558,"WALSH, Justice:",majority,"WALSH, Justice:\nIn this appeal from the Super..."
4,11638241,"S&R ASSOCIATES, L.P., III, a Limited Partnersh...",1998,Delaware Superior Court,725 A.2d 431,"QUILLEN, J.",majority,"QUILLEN, J.\nThis is the Court’s opinion on De..."


Let's use some sample text here:

In [ ]:
text = "Prof. Zurich hailed from Zurich. She got 3 M.A.'s from ETH."

In [ ]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
sentences = sent_tokenize(text)
print(sentences)

['Prof. Zurich hailed from Zurich.', 'She got 3 M.A.', "'s from ETH."]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
sentences = list(doc.sents)
print(sentences)

[Prof. Zurich hailed from Zurich., She got 3 M.A.'s from ETH.]


Change the capitalization of the text:

In [ ]:
# Capitalization
text_lower = text.lower() # go to lower-case

In [ ]:
#####
# Punctuation
#####

# recipe for fast punctuation removal
from string import punctuation
print ("punctuation:", punctuation)
punc_remover = str.maketrans('','',punctuation) 
text_nopunc = text_lower.translate(punc_remover)
print(text_nopunc)


punctuation: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
prof zurich hailed from zurich she got 3 mas from eth


In [ ]:
# Tokens
tokens = text_nopunc.split() # splits a string on white space
print(tokens)

['prof', 'zurich', 'hailed', 'from', 'zurich', 'she', 'got', '3', 'mas', 'from', 'eth']


In [ ]:
# Numbers
# remove numbers (keep if not a digit)
no_numbers = [t for t in tokens if not t.isdigit()]
# keep if not a digit, else replace with "#"
norm_numbers = [t if not t.isdigit() else '#' 
                for t in tokens ]
print(no_numbers )
print(norm_numbers)

['prof', 'zurich', 'hailed', 'from', 'zurich', 'she', 'got', 'mas', 'from', 'eth']
['prof', 'zurich', 'hailed', 'from', 'zurich', 'she', 'got', '#', 'mas', 'from', 'eth']


In [ ]:
# Stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stoplist = stopwords.words('english') 
print ("stop words:", stoplist)
# keep if not a stopword
nostop = [t for t in norm_numbers if t not in stoplist]
print(nostop)

stop words: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# scikit-learn stopwords
# depending on sklearn version, for sklearn==0.24.1, stop_words are here
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS as stop_words
sorted(list(stop_words))[:20]

# in older versions, one may hvae to import it like this
# from sklearn.feature_extraction import stop_words
# sorted(list(stop_words.ENGLISH_STOP_WORDS))[:20]

['a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst']

In [ ]:
from shifterator import shifts as rs
# spacy stopwords
sorted(list(nlp.Defaults.stop_words))[:20]

["'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also']

In [ ]:
# Stemming
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english') # snowball stemmer, english
# remake list of tokens, replace with stemmed versions
tokens_stemmed = [stemmer.stem(t) for t in tokens]
print(tokens_stemmed)

['prof', 'zurich', 'hail', 'from', 'zurich', 'she', 'got', '3', 'mas', 'from', 'eth']


In [ ]:
stemmer = SnowballStemmer('german') # snowball stemmer, german
print(stemmer.stem("Autobahnen"))

autobahn


In [ ]:
# Lemmatizing
nltk.download("wordnet")
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
wnl.lemmatize('corporation'), wnl.lemmatize('corporations')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


('corporation', 'corporation')

Now let's implement this approach on the entire database:

In [ ]:
from string import punctuation
translator = str.maketrans('','',punctuation) 
from nltk.corpus import stopwords
stoplist = set(stopwords.words('english'))
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')

def normalize_text(doc):
    "Input doc and return clean list of tokens"
    doc = doc.replace('\r', ' ').replace('\n', ' ')
    lower = doc.lower() # all lower case
    nopunc = lower.translate(translator) # remove punctuation
    words = nopunc.split() # split into tokens
    nostop = [w for w in words if w not in stoplist] # remove stopwords
    no_numbers = [w if not w.isdigit() else '#' for w in nostop] # normalize numbers
    stemmed = [stemmer.stem(w) for w in no_numbers] # stem each word
    return stemmed


In [ ]:
df.iloc[0:100 , :]['text'].apply(normalize_text)

0     [holland, justic, court, ronald, l, cantrel, “...
1     [per, curiam, appeal, affirm, trial, court, ad...
2     [holland, justic, appeal, employerappel, state...
3     [walsh, justic, appeal, superior, court, requi...
4     [quillen, j, court, opinion, defend, shell, oi...
                            ...                        
95    [opinion, remand, chandler, chancellor, eas, r...
96    [per, curiam, matter, court, disciplinari, pro...
97    [veasey, chief, justic, appeal, statutori, app...
98    [opinion, steel, vice, chancellor, issu, prese...
99    [opinion, lamb, vice, chancellor, action, sect...
Name: text, Length: 100, dtype: object

In [ ]:
from gensim.utils import simple_preprocess
print(simple_preprocess(text))

['prof', 'zurich', 'hailed', 'from', 'zurich', 'she', 'got', 'from', 'eth']


In [ ]:
from collections import Counter
print(Counter(simple_preprocess(text)))

Counter({'zurich': 2, 'from': 2, 'prof': 1, 'hailed': 1, 'she': 1, 'got': 1, 'eth': 1})


In [ ]:
text = 'Science cannot solve the ultimate mystery of nature. And that is because, in the last analysis, we ourselves are a part of the mystery that we are trying to solve.'
nltk.download('averaged_perceptron_tagger')
from nltk.tag import perceptron 
from nltk import word_tokenize
tagger = perceptron.PerceptronTagger()
tokens = word_tokenize(text)
tagged_sentence = tagger.tag(tokens)
tagged_sentence

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('Science', 'NN'),
 ('can', 'MD'),
 ('not', 'RB'),
 ('solve', 'VB'),
 ('the', 'DT'),
 ('ultimate', 'JJ'),
 ('mystery', 'NN'),
 ('of', 'IN'),
 ('nature', 'NN'),
 ('.', '.'),
 ('And', 'CC'),
 ('that', 'DT'),
 ('is', 'VBZ'),
 ('because', 'RB'),
 (',', ','),
 ('in', 'IN'),
 ('the', 'DT'),
 ('last', 'JJ'),
 ('analysis', 'NN'),
 (',', ','),
 ('we', 'PRP'),
 ('ourselves', 'PRP'),
 ('are', 'VBP'),
 ('a', 'DT'),
 ('part', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('mystery', 'NN'),
 ('that', 'IN'),
 ('we', 'PRP'),
 ('are', 'VBP'),
 ('trying', 'VBG'),
 ('to', 'TO'),
 ('solve', 'VB'),
 ('.', '.')]

In [ ]:
#Get list of nouns, adjectives, and verbs from WordNet
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

nouns = set([x.lemma_names()[0].lower() 
             for x in wn.all_synsets('n')])

adjectives = set([x.lemma_names()[0].lower() 
             for x in wn.all_synsets('a')])

verbs = set([x.lemma_names()[0].lower() 
             for x in wn.all_synsets('v')])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
dfs = df.sample(10)
dfs['doc'] = dfs['text'].apply(nlp)

In [ ]:
# The spacy model already gives you sentences and tokens.
# For example:
sent1 = list(dfs['doc'].iloc[0].sents)[1]
sent1 # sentence


John T. Haney, plaintiff, filed a complaint against Felix L. Laub, defendant, alleging that Laub has violated the terms of a Stock Option Agreement.

In [ ]:
# tokens
list(sent1) 

[John,
 T.,
 Haney,
 ,,
 plaintiff,
 ,,
 filed,
 a,
 complaint,
 against,
 Felix,
 L.,
 Laub,
 ,,
 defendant,
 ,,
 alleging,
 that,
 Laub,
 has,
 violated,
 the,
 terms,
 of,
 a,
 Stock,
 Option,
 Agreement,
 .]

In [ ]:
# lemmas
[x.lemma_ for x in sent1]

['John',
 'T.',
 'Haney',
 ',',
 'plaintiff',
 ',',
 'file',
 'a',
 'complaint',
 'against',
 'Felix',
 'L.',
 'Laub',
 ',',
 'defendant',
 ',',
 'allege',
 'that',
 'Laub',
 'have',
 'violate',
 'the',
 'term',
 'of',
 'a',
 'Stock',
 'Option',
 'Agreement',
 '.']

In [ ]:
# POS tags
[x.tag_ for x in sent1]

['NNP',
 'NNP',
 'NNP',
 ',',
 'NN',
 ',',
 'VBD',
 'DT',
 'NN',
 'IN',
 'NNP',
 'NNP',
 'NNP',
 ',',
 'NN',
 ',',
 'VBG',
 'IN',
 'NNP',
 'VBZ',
 'VBN',
 'DT',
 'NNS',
 'IN',
 'DT',
 'NNP',
 'NNP',
 'NNP',
 '.']

In [ ]:
dfs['num_words'] = dfs['doc'].apply(lambda x: len(list(x)))
dfs['num_words'] = dfs['doc'].apply(lambda x: len(list(x.sents)))


print(len(tokens),'words in corpus.')
words_per_sent = len(tokens) / len(sentences)
print(words_per_sent,'words per sentence.')

35 words in corpus.
17.5 words per sentence.


In [ ]:
def tokenize(x, nlp):
    # lemmatize and lowercase without stopwords, punctuation and numbers
    return [w.lemma_.lower() for w in nlp(x) if not w.is_stop and not w.is_punct and not w.is_digit]
tokenize(text, nlp)

['science',
 'solve',
 'ultimate',
 'mystery',
 'nature',
 'analysis',
 'mystery',
 'try',
 'solve']

In [ ]:
from nltk import ngrams
from collections import Counter

# get n-gram counts for 10 documents
grams = []
for i, row in df.iterrows():
    tokens = row['text'].lower().split() # get tokens
    for n in range(2,4):
        grams += list(ngrams(tokens,n)) # get bigrams, trigrams, and quadgrams
    if i > 50:
        break
Counter(grams).most_common()[:8]  # most frequent n-grams

[(('of', 'the'), 2500),
 (('to', 'the'), 924),
 (('that', 'the'), 860),
 (('in', 'the'), 823),
 (('by', 'the'), 470),
 (('on', 'the'), 422),
 (('of', 'a'), 400),
 (('and', 'the'), 358)]

# Tokenizers

In [ ]:
# Counter is a quick pure-python solution.
from collections import Counter
freqs = Counter(tokens)
freqs.most_common()[:20]

[('the', 40),
 ('to', 37),
 ('of', 25),
 ('in', 21),
 ('§', 18),
 ('a', 16),
 ('court', 14),
 ('and', 14),
 ('alimony', 13),
 ('as', 13),
 ('be', 13),
 ('husband’s', 11),
 ('pension', 11),
 ('under', 9),
 ('v.', 9),
 ('court’s', 8),
 ('was', 8),
 ('that', 8),
 ('award', 7),
 ('its', 7)]

In [ ]:
# tf-idf vectorizer up-weights rare/distinctive words
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=0.01, 
                        max_df=0.9,  
                        max_features=1000,
                        stop_words='english',
                        use_idf=True, # the new piece
                        ngram_range=(1,2))

X_tfidf = tfidf.fit_transform(df['text'])
pd.to_pickle(X_tfidf,'X_tfidf.pkl')

# Customer tokenizers and stemmers

In [ ]:
# hash vectorizer
from sklearn.feature_extraction.text import HashingVectorizer

hv = HashingVectorizer(n_features=10)
X_hash = hv.fit_transform(df['text'])
X_hash

<16981x10 sparse matrix of type '<class 'numpy.float64'>'
	with 163575 stored elements in Compressed Sparse Row format>

# Debugging hasing vectorizer

In [ ]:
!pip install eli5
from eli5.sklearn import InvertableHashingVectorizer

ivec = InvertableHashingVectorizer(hv)
inverted_hv = ivec.fit(df['text'])
print ([i for i in inverted_hv.get_feature_names()])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Hashing with Keras

In [ ]:
# !pip install keras
import tensorflow as tf
import keras
text = "Prof. Zurich hailed from Zurich. She got 3 M.A.'s from ETH."
n = 5 # num features
tf.keras.preprocessing.text.hashing_trick(
    text, n, hash_function=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True, split=' '
)

[2, 4, 1, 3, 4, 1, 1, 1, 1, 4, 2, 3, 3]

# Collocations: Point-Wise Mutual Information

In [ ]:
from operator import mul
from functools import reduce
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')

def get_gmean(phrase, termfreqs):
    words = phrase.split('_')
    n = len(words)
    p = [termfreqs[w]**(1/n) for w in words]
    print (words, n, p)
    numerator = termfreqs[phrase]   
    denominator = reduce(mul, p)
    print (numerator, denominator)
    if denominator == 0:
        return 0
    gmean = numerator / denominator
    return gmean

# POS-filtered N-grams

In [ ]:


from collections import Counter

# Normalized Penn tags
tagdict = { 'NN':'N',
            'NNS':'N',
                                    
            'JJ':'A',
            'JJR':'A',
            'JJS':'A',
            'VBG':'A', # gerunds/participles treated like adjectives

            'RB':'A', # adverbs treated as adjectives
            'RBR':'A',
            'RBS':'A',
            'PDT':'A', # predeterminer            

            'VB':'V',
            'VBD':'V',
            'VBN':'V',
            'VBP':'V',
            'VBZ':'V',
            'MD': 'V', # modals treated as verbs
            'RP': 'V', # particles treated as verbs
            
            'DT':'D',
                        
            'IN':'P',
            'TO':'P',

            'CC': 'C'}

tagpatterns = {'A','N','V','P','C','D',
           'AN','NN', 'VN', 'VV', 
            #'NV',
            'VP',                                    
            'NNN','AAN','ANN','NAN','NPN',
            'VAN','VNN', 'AVN', 'VVN',
            'VPN', 'VDN', 
            #'ANV','NVV','VVV', 'NNV',
            'VVP','VAV','VVN',
            'NCN','VCV', 'ACA',  
            'PAN',
            'NCVN','ANNN','NNNN','NPNN', 'AANN' 'ANNN','ANPN','NNPN','NPAN', 
            'ACAN', 'NCNN', 'NNCN', 'ANCN', 'NCAN',
            'PDAN', 'PNPN',
            'VDNN', 'VDAN','VVDN'}

def count_pos_grams(sentence, max_phrase_length=4):
    sent_freq = Counter()
    tagwords = []
    for (word,tag) in tagger.tag(sentence):
        if tag in tagdict:
            normtag = tagdict[tag]
            stemmed = word.lower()#stemmer.stem(word)
            tagwords.append((stemmed,normtag))
        else:
            tagwords.append(None)
    for n in range(1,max_phrase_length+1):            
        rawgrams = ngrams(tagwords,n)
        for rawgram in rawgrams:
            # skip grams that have rare words
            
            if None in rawgram:
                continue
            gramtags = ''.join([x[1][0] for x in rawgram])
            if gramtags in tagpatterns:
                 # if tag sequence is allowed, add to counter
                gram = '_'.join([x[0] for x in rawgram])
                sent_freq[gram] += 1
    return sent_freq


In [ ]:
max_phrase_length = 4
termfreqs = Counter()

for i, doc in dfs.iterrows():    
    termfreqs.update(count_pos_grams(nltk.word_tokenize(doc['text'])))

In [ ]:
# filter out unigrams
grams = [x for x in termfreqs.most_common() if '_' in x[0]]
# make dataframe of geometric mean associations for each gram
gmeans = pd.DataFrame([(gram[0], get_gmean(gram[0],termfreqs)) for gram in grams],
              columns=['ngram','gmean'])
gmeans.sort_values('gmean',ascending=False)

Streaming output truncated to the last 5000 lines.
['to', 'a', 'final', 'liquidation'] 4 [6.066031568162846, 5.066655238516518, 1.7320508075688772, 1.189207115002721]
1 63.3058939675851
['standing', 'and', 'general', 'condition'] 4 [1.7782794100389228, 5.778547889749466, 2.114742526881128, 1.4953487812212205]
1 32.49516277997609
['inspect', 'the', 'general', 'books'] 4 [1.0, 7.698190018446947, 2.114742526881128, 2.23606797749979]
1 36.40249307229102
['as', 'the', 'other', 'members'] 4 [4.154837722659222, 7.698190018446947, 2.9718278662008415, 1.8988289221159418]
1 180.4895996333257
['had', 'made', 'no', 'settlement'] 4 [3.281818034911291, 3.288868167986058, 3.589442676339201, 2.5755095769013945]
1 99.78175859926667
['make', 'up', 'the', 'accounts'] 4 [1.9679896712654303, 2.1657367706679937, 7.698190018446947, 3.1622776601683795]
1 103.75692966176011
['could', 'get', 'a', 'settlement'] 4 [3.11373725847777, 1.3160740129524924, 5.066655238516518, 2.5755095769013945]
1 53.47450829275081
['

,ngram,gmean
545,scouring_pad,1.000000
523,social_worker,1.000000
364,numerous_uncertainties,1.000000
586,calf_atrophy,1.000000
543,digital_scale,1.000000
...,...,...
4337,is_the_contract,0.002209
7708,is_on,0.002078
4821,is_the_case,0.001893
2827,be_in,0.001826


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
token_vec = CountVectorizer(min_df=0.01, # at min 1% of docs
                            max_df=.9,  # no tokens appearing in more than 90% of all docs
                            max_features=10000,                                                
                            preprocessor =lambda x: x, # for tokens
                            tokenizer = lambda x: x, # for tokens
                            stop_words='english',
                            ngram_range=(1,1))
#print (df["tokens_cleaned"][0])
#print (df["opinion_text"][0])
#print (df.columns)
X_pos = token_vec.fit_transform(df['text'])
X_pos

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  % sorted(inconsistent)


<16981x78 sparse matrix of type '<class 'numpy.int64'>'
	with 644734 stored elements in Compressed Sparse Row format>

In [ ]:
# spacy noun chunks for the first document
i = 0
for chunk in nlp(dfs.iloc[0]['text']).noun_chunks:
    print ('{} - {}'.format(chunk, chunk.label_))
    if i > 10:
        break
    i += 1

OPINION - NP
SUMMARY - NP
Judge - NP
John T. Haney - NP
plaintiff - NP
a complaint - NP
Felix L. Laub - NP
defendant - NP
Laub - NP
the terms - NP
a Stock Option Agreement - NP
The complaint - NP


In [ ]:
# spacy prepositional phrase chunking
# find all PPs (e.g. "in the court")
def get_pps(text):
    doc = nlp(text)
    for token in doc:
        # if we have a prepositional object and the dependency head of the current token is a preposition, we have a prepositional phrase
        if token.dep_ == "pobj" and token.head.dep_ == "prep":
            # we just iterate through the subtree then and collect the dependency head, the token itself and all tokens in the subtree
            pp = token.head.text + " " + ' '.join([tok.orth_ for tok in token.subtree])
            print (pp)
            
get_pps(df["text"][0])

before it
for a writ of habeas corpus and the State of Delaware
of habeas corpus and the State of Delaware
of Delaware
to the Court
in civil contempt
for failure to pay child support
to an indefinite period of level V incarceration
of level V incarceration
until such time as Cantrell reduced his arrears by $ 2000
by $ 2000
v. Salicete , Del. Supr
of the sentence
for level IV home confinement
in violation of the conditions of his home confinement
of the conditions of his home confinement
of his home confinement
at level V incarceration
pending the availability of bed space at a level IV halfway house
of bed space
at a level IV halfway house
for an original writ of habeas corpus with this Court
of habeas corpus with this Court
with this Court
on June 11,1996
to June 5 , 1996
for contempt
of habe - as carpus
in the Superior Court
under 10 Del. C. §
under 6903
v. DCSE / Durkin , 672 A.2d 9 , 11 ( Del.1995 )
In response to this Court ’s holding in Newman , 10 Del. C. § § 6901 and 6903(a )
t

# Feature Selection

In [ ]:
df.head()

,id,name,decision_date,court,citations,author,type,text
0,11737169,In the Matter of Ronald L. CANTRELL for a Writ...,1996,Delaware Supreme Court,678 A.2d 525,"HOLLAND, Justice:",majority,"HOLLAND, Justice:\nThe Court has before it Ron..."
1,11684804,"Hendrik LIKET, Defendant Below, Appellant, v. ...",1998,Delaware Supreme Court,719 A.2d 935,PER CURIAM.,majority,"PER CURIAM.\nIn this appeal, we affirm the tri..."
2,11684732,"STATE of Delaware, Employer Below, Appellant, ...",1998,Delaware Supreme Court,719 A.2d 930,"HOLLAND, Justice:",majority,"HOLLAND, Justice:\nThis is an appeal by the em..."
3,11637715,"Eric BARKLEY, Defendant Below, Appellant, v. S...",1999,Delaware Supreme Court,724 A.2d 558,"WALSH, Justice:",majority,"WALSH, Justice:\nIn this appeal from the Super..."
4,11638241,"S&R ASSOCIATES, L.P., III, a Limited Partnersh...",1998,Delaware Superior Court,725 A.2d 431,"QUILLEN, J.",majority,"QUILLEN, J.\nThis is the Court’s opinion on De..."


In [ ]:
#%% Univariate feature selection using chi2
from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression, f_classif, mutual_info_classif
select = SelectKBest(chi2, k=10)
Y = df['citations']
X_new = select.fit_transform(X_pos, Y)
# top 10 features by chi-squared:
vocab = vec.get_feature_names()
[vocab[i] for i in np.argsort(select.scores_)[:10]]

NameError: ignored

In [ ]:
#%% top 10 features by  ANOVA F-value:
select = SelectKBest(f_classif, k=10)
select.fit(X, Y)
[vocab[i] for i in np.argsort(select.scores_)[:10]]

In [ ]:
#%% top 10 features by linear regression
select = SelectKBest(f_regression, k=10)
select.fit(X, Y)
[vocab[i] for i in np.argsort(select.scores_)[:10]]

In [ ]:
#%% top 10 features by mutual information (classification)
select = SelectKBest(mutual_info_classif, k=10)
select.fit(X[:1000], Y[:1000])
[vocab[i] for i in np.argsort(select.scores_)[:10]]

# Sentencepiece Tokenization

In [ ]:
!pip install sentencepiece

In [ ]:
import sentencepiece as spm
# training spm requires a text file as input, so generate a small one
with open("sample_text.txt", "w") as outfile:
        for text in df["text"][:100]:
            outfile.write(text + "\n")
            
spm.SentencePieceTrainer.train(input="sample_text.txt", model_prefix='m', vocab_size=100)

In [ ]:
df["text"][0][:100]

In [ ]:
# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')
sp.encode_as_pieces(df["text"][0][:100])

In [ ]:
print(sp.encode_as_ids(df["text"][0][:100]))

# Part 3 - Observing differences between texts

**Part 3.a : Scattertext using spacy**
Scattertext to visualize text comparisons. You can take a case from Delaware court of Chancery vs Delaware Criminal cour for example - https://github.com/JasonKessler/scattertext

In [ ]:
!pip install scattertext
import scattertext as st

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
concat['case_parse'] = concat.case_text.apply(st.whitespace_nlp_with_sentences)

In [ ]:
corpus = st.CorpusFromParsedDocuments(
    concat, category_col='court_name', parsed_col='case_parse'
).build().get_unigram_corpus().compact(st.AssociationCompactor(2000))

In [ ]:
html = st.produce_scattertext_explorer(
    corpus,
    category='Delaware Court of Chancery', category_name='Delaware Court of Chancery', not_category_name='Delaware Court of Common Pleas',
    minimum_term_frequency=0, pmi_threshold_coefficient=0,
    width_in_pixels=1000, metadata=corpus.get_df()['name'],
    transform=st.Scalers.dense_rank
)

In [ ]:
open('./case.html', 'w').write(html)

In [ ]:
import IPython
IPython.display.HTML(filename='./case.html')

## Part 3.b: Shifterator

In [ ]:
def clean_text(txt):
    tmp = [t.lower().split() for t in txt]
    tmp = list(itertools.chain(*tmp))
    tmp = collections.Counter(tmp)
    return tmp

In [ ]:
clean_texts_chancery = clean_text(concat[concat['court_name']=='Delaware Court of Chancery']['case_text'].to_list())
clean_texts_criminal = clean_text(concat[concat['court_name']=='Delaware Court of Common Pleas']['case_text'].to_list())

In [ ]:
entropy_shift = rs.EntropyShift(type2freq_1=clean_texts_chancery, type2freq_2=clean_texts_criminal, base=2)
entropy_shift.get_shift_graph() 